In [65]:
import urllib.request
import json

# https://www.mediawiki.org/wiki/API:Query
SEARCH_URL = "https://de.wiktionary.org/w/api.php?action=query&format=json&redirects&titles={}"


def request_json_from_url(url):
    try:
        with urllib.request.urlopen(url) as response:
            content = response.read().decode('utf-8')
            return json.loads(content)
    except urllib.error.URLError as e:
        print("Failed to fetch HTML from the URL:", e)    


def find_word_pageid(text):
    url = SEARCH_URL.format(text)
    response = request_json_from_url(url)
    pages = list(response["query"]["pages"].keys())
    if not pages:
        return None
    return int(pages[0])
    
page_id = find_word_pageid("bestellen")
page_id

76132

In [71]:
import re
from rich import print as rprint

# https://www.mediawiki.org/wiki/API:Parsing_wikitext
PAGE_URL = "https://de.wiktionary.org/w/api.php?redirects&action=parse&format=json&prop=wikitext&pageid={}"
AUDIO_RE = re.compile(r"\{\{Audio\|(.*?)\}\}")

def get_audio_file_name(pageid):
    url = PAGE_URL.format(pageid)
    response = request_json_from_url(url)
    wikitext = response["parse"]["wikitext"]["*"]
    rprint(wikitext)
    matches = AUDIO_RE.findall(wikitext)
    if not matches:
        return
    return matches[0]

audio_file_name = get_audio_file_name(page_id)
audio_file_name

== bestellen ({{Sprache|Deutsch}}) ==
=== {{Wortart|Verb|Deutsch}} ===

{{Deutsch Verb Übersicht
|Präsens_ich=bestelle
|Präsens_du=bestellst
|Präsens_er, sie, es=bestellt
|Präteritum_ich=bestellte
|Konjunktiv II_ich=bestellte
|Imperativ Singular=bestell
|Imperativ Singular*=bestelle
|Imperativ Plural=bestellt
|Partizip II=bestellt
|Hilfsverb=haben
}}

{{Worttrennung}}
:be·stel·len, {{Prät.}} be·stell·te, {{Part.}} be·stellt

{{Aussprache}}
:{{IPA}} {{Lautschrift|bəˈʃtɛlən}}
:{{Hörbeispiele}} {{Audio|De-bestellen.ogg}}, {{Audio|De-bestellen2.ogg}}
:{{Reime}} {{Reim|ɛlən|Deutsch}}

{{Bedeutungen}}
:[1] eine Ware oder [[Dienstleistung]] zur [[Lieferung]] in Auftrag geben
:[2] eine Nachricht ausrichten, überbringen
:[3] {{K|Landwirtschaft}} den Boden bearbeiten
:[4] {{K|ft=jemanden bestellen zu}} jemandem ein Amt (einen [[Dienstposten]]) übertragen
:{{QS Bedeutungen|fehlen}}

{{Herkunft}}
:[] ''bistellen,'' [] ''bestellen'' „[], [], []“; von ''anordnen'' leitet sich die heutige Bedeutung „in Auftrag 
geben“ ab. Siehe hierzu auch unter ''[].''<ref>{{Lit-Kluge: Etymologisches Wörterbuch|A=24}} Seite 115</ref>

{{Synonyme}}
:[1] [], [], [], []
:[2] [], [[überbringen]]
:[3] [], [], []
:[4] []

{{Gegenwörter}}
:[1] []
:[3] [] liegenlassen
:[4] []

{{Beispiele}}
:[1] Ich habe mir eben eine Pizza ''bestellt.''
:[1] Ich habe beim Italiener für heute Abend einen Tisch ''bestellt.''
:[1] Könnten Sie mir bitte ein Taxi ''bestellen?''
:[1] „Lucy nahm eines der aquamarinblauen Telefone und ''bestellte'' Kaffee und etwas zu essen.“<ref>{{Literatur | 
Autor=Mario Puzo | Titel=Der Pate | Verlag=Rowohlt Taschenbuch Verlag | Ort=Reinbek bei Hamburg | Jahr=1990 | 
ISBN=3-499-11442-9 | Seiten=332 }}</ref>
:[2] Kannst du ihm einen schönen Gruß ''bestellen?''
:[2] ''Bestell'' ihr, sie kann mich mal!
:[3] Im Frühjahr werden die Äcker ''bestellt.''
:[3] Die UN warnen vor einer Hungersnot im Bürgerkriegsland Südsudan. Mehr als eine Million Menschen sind auf der 
Flucht und konnten ihre Felder nicht ''bestellen.''<ref>{{Per-Deutsche Welle | 
Online=http://www.dw.de/hungersnot-bedroht-s%C3%BCdsudan/a-17820424 | Autor=Sarah Steffen / Friederike Müller | 
Titel=Südsudan - Hungersnot bedroht Südsudan | Tag=01 | Monat=08 | Jahr=2014 | Zugriff=2015-03-07 }}</ref>
:[4] „Der Abgeordnete Noonan wurde, 18 Monate nach seinem Einzug ins Parlament, zum Justizminister ''bestellt,'' 
anschließend führte er kurz das Wirtschaftsministerium.“<ref>Frankfurter Allgemeine Zeitung, 15. Juli 2011, Seite 8
Spalte 6</ref>

{{Redewendungen}}
:[]

{{Charakteristische Wortkombinationen}}
:[1] einen [[Tisch]] ''bestellen'', ein [[Produkt]]/[[Taxi]] ''bestellen'', eine [[Ware]] ''bestellen''
:[2] einen [[Gruß]] ''bestellen'', beste [[Wünsche]] ''bestellen''

{{Wortbildungen}}
:''Adjektive:'' []
:''Substantive:'' [[Bestellen]], [[Bestellformular]], [[Bestellkarte]], [[Bestellmenge]], [[Bestellmöglichkeit]], 
[[Bestellnummer]], [[Bestellschein]], [[Bestellung]]
:''Verben:'' [], [], [], [], [], [], [], []

==== {{Übersetzungen}} ====
{{Ü-Tabelle|1|G=eine Ware oder Dienstleistung zur Lieferung in Auftrag geben|Ü-Liste=
*{{da}}: {{Ü|da|bestille}}
*{{en}}: {{Ü|en|order}}
*{{fr}}: {{Ü|fr|commander}}, {{Ü|fr|réserver}}
*{{el}}: {{Üt|el|παραγγέλνω|parangélno}}
*{{ia}}: {{Ü|ia|ordinar}}
*{{it}}: {{Ü|it|ordinare}}
*{{ja}}: {{Üt|ja|注文する|ちゅうもんする, chûmon suru}}
*{{ca}}: {{Ü|ca|encomanar}}, {{Ü|ca|encarregar}}
*{{nl}}: {{Ü|nl|bestellen}}
*{{no}}: {{Ü|no|bestille}}
*{{pl}}: {{Ü|pl|zamówić}}
*{{pt}}: {{Ü|pt|solicitar}}, {{Ü|pt|encomendar}}
*{{ro}}: {{Ü|ro|comanda}}, {{Ü|ro|ordona}}
*{{ru}}: {{Üt|ru|заказывать}}
*{{sv}}: {{Ü|sv|beställa}}, {{Ü|sv|rekvirera}}
*{{es}}: {{Ü|es|encargar}}, {{Ü|es|pedir}}
*{{cs}}: {{Ü|cs|objednat}}
*{{uk}}: {{Üt|uk|замовити|}}
*{{hu}}: {{Ü|hu|megrendel}}
*{{be}}: {{Üt|be|замовіць|}}
}}

{{Ü-Tabelle|2|G=eine Nachricht ausrichten, überbringen|Ü-Liste=
*{{en}}: {{Ü|en|convey}}
*{{fr}}: {{Ü|fr|transmettre}}
*{{ja}}: {{Üt|ja|伝える|つたえる, tsutaeru}}
*{{pt}}: {{Ü|pt|pedir}}
*{{ru}}: {

'De-bestellen.ogg'

In [70]:
from rich import print as rprint

# https://www.mediawiki.org/wiki/API:Parsing_wikitext
FILES_URL = "https://de.wiktionary.org/w/api.php?action=query&format=json&redirects&prop=imageinfo&iiprop=url&titles=File:{}"

def get_file_url(file_name):
    url = FILES_URL.format(file_name)
    response = request_json_from_url(url)
    pages = list(response["query"]['pages'].values())
    imageinfo = pages[0]["imageinfo"][0]
    return imageinfo["url"]

get_file_url(audio_file_name)

'https://upload.wikimedia.org/wikipedia/commons/9/9b/De-bestellen.ogg'

In [23]:
!poetry install

Installing dependencies from lock file

Package operations: 20 installs, 5 updates, 0 removals

  • Installing blinker (1.7.0)
  • Installing click (8.1.7)
  • Downgrading idna (3.7 -> 3.6)
  • Downgrading importlib-metadata (7.1.0 -> 7.0.2)
  • Installing itsdangerous (2.1.2)
  • Installing pyqt6-qt6 (6.6.2)
  • Installing pyqt6-sip (13.6.0)
  • Installing pysocks (1.7.1)
  • Downgrading referencing (0.35.0 -> 0.34.0)
  • Installing werkzeug (3.0.1)
  • Installing flask (3.0.2)
  • Installing markdown (3.6)
  • Installing orjson (3.9.15)
  • Installing pyqt6 (6.6.1)
  • Installing pyqt6-webengine-qt6 (6.6.2)
  • Installing protobuf (5.26.0)
  • Installing anki (23.12.1)
  • Installing flask-cors (4.0.0)
  • Installing mypy-extensions (1.0.0)
  • Downgrading send2trash (1.8.3 -> 1.8.2)
  • Downgrading typing-extensions (4.11.0 -> 4.10.0)
  • Installing waitress (3.0.0)
  • Installing pyqt6-webengine (6.6.0)
  • Installing aqt (23.12.1)
  • Installing mypy (1.9.0)

Installing the curren

The current project could not be installed: No file/folder found for package anki-quick-color-addon
If you do not want to install the current project use --no-root
